In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal
import random

In [2]:
aa_notes = pd.read_csv('aa_notes.csv', sep=',',header=0)
# Generate amino acid note
def aa_note(aa,length):
    row = np.where(aa_notes['Amino Acid'] == aa)[0][0]
    wave = np.zeros(int(44100 * float(length)))
    harm = 0
    for freq in aa_notes.iloc[row,1:]:
        if freq == 0:
            break
        else:
            # Area for experimentation
            volume = np.exp(-harm)
            
            
            phases = np.cumsum(2.0 * np.pi * freq / 44100 * np.ones(int(44100 * float(length))))
            wave += np.sin(phases) * volume # sine wave
            
            # Area for experimentation
            if harm > 2:
                wave += scipy.signal.sawtooth(phases) * volume              # sawtooth wave
            elif harm > 2:
                wave += scipy.signal.square(phases) * volume                # square wave
            elif harm > 2:
                wave += scipy.signal.sawtooth(phases,width=0.5) * volume    # triangle wave
                
                
        harm += 1
    return wave
  
# Filter sound wave with simple envelope
def envelope(unfiltered_wave,length):
    #Attack
    attack_phases = np.cumsum(2.0 * np.pi * (1/2)*length / 44100 * np.ones(int(44100 * float(length))))
    attack = np.log10(attack_phases)
    attack = (attack - attack.min())/(attack.max()-attack.min())
    #Sustain
    sus_phases = np.cumsum(2.0 * np.pi * (1/(2**length)) / 44100 * np.ones(int(44100 * float(length))))
    sus = np.sin(sus_phases)*0.5
    #Release
    rel_phases = np.cumsum(2.0 * np.pi * 1/(4*length) / 44100 * np.ones(int(44100 * float(length))))
    rel = np.cos(rel_phases)*0.5

    return (((unfiltered_wave*attack)+(unfiltered_wave*sus))*rel)

In [3]:
# Protein Sequences
seq1 = 'MQIFVKTLT'
seq2 = 'KEGIPPKEG'

In [4]:
# Duration of each note
unit_length = 0.2
# Generate melodies
melody1 = []
for i in range(len(seq1)):
    residue = seq1[i]   
    sound_wave = aa_note(residue,unit_length)
    env_wave = envelope(sound_wave,unit_length)
    melody1 = melody1 + env_wave.tolist()
melody1 = np.asarray(melody1)
melody2 = []
for i in range(len(seq2)):
    residue = seq2[i]   
    sound_wave = aa_note(residue,unit_length)
    env_wave = envelope(sound_wave,unit_length)
    melody2 = melody2 + env_wave.tolist()
melody2 = np.asarray(melody2)

In [5]:
complete_music = melody1 + melody2
#rescale to a bit less thanrange [-1,1]
complete_music = 1.8*(complete_music - complete_music.min())/np.ptp(complete_music)-0.9

In [6]:
import sounddevice as sd

In [7]:
sd.play(complete_music, 44100)

In [6]:
import wave, struct
sampleRate = 44100.0 # hertz
duration = 1.0 # seconds
frequency = 440.0 # hertz
obj = wave.open('sound.wav','w')
obj.setnchannels(1) # mono
obj.setsampwidth(2)
obj.setframerate(sampleRate)
for i in range(len(complete_music)):
    value = int(complete_music[i] * 32767)
    data = struct.pack('<h', value)
    obj.writeframesraw( data )
obj.close()